In [22]:
from typing import Dict, List, Tuple, Union, TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Define state schema
class State(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    attention_score: int
    consecutive_wrong: int
    user_verified: bool
    next: str
    iteration: int

# Initialize LLM
llm = ChatOpenAI(
    temperature=0.7,
    api_key=os.getenv('OPENAI_API_KEY')
)

class LLMWorkflow:
    def __init__(self):
        self.workflow = self._create_workflow()
    
    @staticmethod    
    def end_function(state: State) -> bool:
        """Determine if workflow should end"""
        conditions = [
            state.get('attention_score', 0) <= 0,
            state.get('consecutive_wrong', 0) >= 3,
            state.get('iteration', 0) >= 10,  # Maximum iterations
            state.get('next') == 'end'
        ]
        return any(conditions)
        
    def _create_workflow(self):
        def greeting(state: State) -> Dict:
            if 'messages' not in state:
                state['messages'] = []
                
            state['messages'].append(AIMessage(content=
                "Welcome! I'm an AI assistant ready to help you. "
                "Before we begin, please tell me your name and why you're here today."
            ))
            state['attention_score'] = 100
            state['consecutive_wrong'] = 0
            state['user_verified'] = False
            state['next'] = 'verify'
            state['iteration'] = 0
            return state

        def verify_user(state: State) -> Dict:
            state['iteration'] = state.get('iteration', 0) + 1
            last_message = state['messages'][-1].content if state['messages'] else ""
            
            response = llm.invoke("Is this response genuine and focused? Answer PASS or FAIL: " + last_message)
            
            if "FAIL" in response.content.upper():
                state['attention_score'] = state.get('attention_score', 100) - 20
                state['messages'].append(AIMessage(content=
                    "I notice some concerns. Could you please respond more naturally?"
                ))
            else:
                state['user_verified'] = True
                state['messages'].append(AIMessage(content=
                    "Great! You seem ready to proceed. Let's continue."
                ))
                
            if self.end_function(state):
                state['next'] = 'end'
                state['messages'].append(AIMessage(content=
                    "Let's conclude our session here. Thank you for participating!"
                ))
            else:
                state['next'] = 'process'
            return state

        def process_response(state: State) -> Dict:
            state['iteration'] = state.get('iteration', 0) + 1
            last_message = state['messages'][-1].content if state['messages'] else ""
            
            response = llm.invoke("Does this show understanding? Answer CORRECT or INCORRECT: " + last_message)
            
            if "CORRECT" in response.content.upper():
                state['consecutive_wrong'] = 0
                state['messages'].append(AIMessage(content=
                    "Excellent! Your response shows good understanding. Let's continue."
                ))
            else:
                state['consecutive_wrong'] = state.get('consecutive_wrong', 0) + 1
                state['messages'].append(AIMessage(content=
                    f"That's not quite right. This is attempt {state['consecutive_wrong']} out of 3."
                ))
                
            if self.end_function(state):
                state['next'] = 'end'
                state['messages'].append(AIMessage(content=
                    "Let's conclude our session here. Thank you for participating!"
                ))
            else:
                state['next'] = 'verify'
            return state

        # Create workflow
        workflow = StateGraph(State)
        
        # Add nodes
        workflow.add_node("greeting", greeting)
        workflow.add_node("verify", verify_user)
        workflow.add_node("process", process_response)

        # Add conditional edges
        def router(state: State) -> str:
            if self.end_function(state):
                return END
            return state["next"]

        for node in ["greeting", "verify", "process"]:
            workflow.add_conditional_edges(
                node,
                router
            )

        # Set entry point
        workflow.set_entry_point("greeting")
        
        return workflow.compile()

    def run(self, user_input: str) -> Dict:
        """Run the workflow with user input"""
        initial_state: State = {
            "messages": [HumanMessage(content=user_input)],
            "attention_score": 100,
            "consecutive_wrong": 0,
            "user_verified": False,
            "next": "greeting",
            "iteration": 0
        }
        
        try:
            final_state = self.workflow.invoke(initial_state)
            return {
                "messages": final_state["messages"],
                "attention_score": final_state.get("attention_score", 0),
                "session_active": not self.end_function(final_state)
            }
        except Exception as e:
            print(f"Error executing workflow: {str(e)}")
            return {
                "messages": [AIMessage(content="Sorry, there was an error processing your input.")],
                "attention_score": 0,
                "session_active": False
            }

# Example usage in Jupyter notebook:
"""
# First create a .env file with:
OPENAI_API_KEY=your-api-key-here

# Then in your notebook:
workflow = LLMWorkflow()

# Test single interaction
result = workflow.run("Hi, I'm Alex and I'm here to learn.")
print(result["messages"][-1].content)

# Or run interactive loop
while result["session_active"]:
    user_input = input("Your response: ")
    result = workflow.run(user_input)
    print(result["messages"][-1].content)
"""

'\n# First create a .env file with:\nOPENAI_API_KEY=your-api-key-here\n\n# Then in your notebook:\nworkflow = LLMWorkflow()\n\n# Test single interaction\nresult = workflow.run("Hi, I\'m Alex and I\'m here to learn.")\nprint(result["messages"][-1].content)\n\n# Or run interactive loop\nwhile result["session_active"]:\n    user_input = input("Your response: ")\n    result = workflow.run(user_input)\n    print(result["messages"][-1].content)\n'

In [23]:
workflow = LLMWorkflow()

# Test with a single interaction
result = workflow.run("Hi, I'm Alex and I'm here to learn.")
print(result["messages"][-1].content)

# Or run an interactive loop
while result["session_active"]:
    user_input = input("Your response: ")
    result = workflow.run(user_input)
    print(result["messages"][-1].content)

Let's conclude our session here. Thank you for participating!
